In [25]:
#Imports
import os, sys
path_to_package = os.path.abspath(os.path.join('../..'))
if path_to_package not in sys.path:
    sys.path.append(path_to_package)


import scipy.stats as stats
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from src.io import *
from src.anova import *
from src.utils import *
from src.visualization import *

In [26]:
ANOVA_DATASET_PATH = "C:\\Users\\Hector\\Desktop\\code\\sc_regmod\\dataset\\anova"

In [6]:
filepaths = get_filepaths(ANOVA_DATASET_PATH)
dataset = []
for filepath in filepaths:
    filename = get_filename(filepath=filepath)
    data = {}
    data["filepath"] = filepath
    data["filename"] = filename
    data["data"] = read_phylip_file(filepath)
    
    dataset.append(data)

In [31]:
# Get columns size by file
def get_columns_size_by_file(dataset):
    res = []
    for data in dataset:
        d = {}
        d["filename"] = data["filename"]
        d["alignment_length"] = data["data"].get_alignment_length()

        res.append(d)
    
    df = pd.DataFrame(res)

    return df

res_1 = get_columns_size_by_file(dataset)
res_1 = res_1.sort_values(by="alignment_length", ascending=False)



In [49]:
# Figura Largo de cadena vs nombre archivo TODOS
fig = px.bar(res_1, x="filename", y="alignment_length")
fig.update_traces(marker_color='red')
fig.show()

In [43]:
# Figura Largo de cadena vs nombre archivo TOP 50
fig = px.bar(res_1.head(50), x="filename", y="alignment_length")
fig.show()